In [28]:
%pylab inline
from collections import defaultdict, OrderedDict
import warnings
import gffutils
import pybedtools
import pandas as pd
import copy
import os
import re
from gffutils.pybedtools_integration import tsses
from copy import deepcopy
from collections import OrderedDict, Callable
import errno

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))


Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [29]:
gtf = '/home/cmb-panasas2/skchoudh/genomes/mm10/annotation/gencode.vM11.annotation.gtf'
gtf_db = '/home/cmb-panasas2/skchoudh/genomes/mm10/annotation/gencode.vM11.annotation.gtf.db'


In [30]:
def create_gene_dict(db):
    '''
    Store each feature line db.all_features() as a dict of dicts
    '''
    gene_dict = DefaultOrderedDict(lambda: DefaultOrderedDict(lambda: DefaultOrderedDict(list)))
    for line_no, feature in enumerate(db.all_features()):
        gene_ids = feature.attributes['gene_id']
        feature_type = feature.featuretype
        if feature_type == 'gene':
            if len(gene_ids)!=1:
                logging.warning('Found multiple gene_ids on line {} in gtf'.format(line_no))
                break
            else:
                gene_id = gene_ids[0]
                gene_dict[gene_id]['gene'] = feature
        else:
            transcript_ids = feature.attributes['transcript_id']

            for gene_id in gene_ids:
                for transcript_id in transcript_ids:
                    gene_dict[gene_id][transcript_id][feature_type].append(feature)
    return gene_dict

In [31]:
db = gffutils.FeatureDB(gtf_db, keep_order=True)
gene_dict = create_gene_dict(db)


In [32]:
for x in db.featuretypes():
    print(x)

CDS
Selenocysteine
UTR
exon
gene
start_codon
stop_codon
transcript


In [43]:
gene_wise_CCDS_dict = DefaultOrderedDict(list)

for line_no, feature in enumerate(db.all_features()):
    gene_ids = feature.attributes['gene_id']    
    feature_type = feature.featuretype
    try:
        tags = feature.attributes['tag']
        if 'CCDS' not in tags:
            # Anything that is not CCDS is not interesting
            continue
        if feature_type != 'transcript':
            continue
        else:
            assert len(gene_ids) == 1
            gene_id = gene_ids[0]
            transcript_ids = feature.attributes['transcript_id']
            assert len(transcript_ids) == 1
            transcript_id = transcript_ids[0]

            feature.attributes['gene_id'] = '{}_{}'.format(gene_id, transcript_id)
            gene_wise_CCDS_dict[gene_id].append(feature)
    except:
        continue

In [44]:
def get_gene_list(gene_dict):
    return list(set(gene_dict.keys()))

def create_bed(regions, bedtype='0'):
    '''Create bed from list of regions
    bedtype: 0 or 1
        0-Based or 1-based coordinate of the BED
    '''
    bedstr = ''
    for region in regions:
        assert len(region.attributes['gene_id']) == 1
        ## GTF start is 1-based, so shift by one while writing 
        ## to 0-based BED format
        if bedtype == '0':
            start = region.start - 1
        else:
            start = region.start
        bedstr += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(region.chrom,
                                             start,
                                             region.stop,
                                             re.sub('\.\d+', '', region.attributes['gene_id'][0]),
                                             '.',
                                             region.strand)
    return bedstr

In [45]:
ccds_bed = ''

for gene_id in get_gene_list(gene_dict):
    ccds = gene_wise_CCDS_dict[gene_id]
    ccds = list(sorted(list(ccds), key=lambda x: x.start))    
    ccds_bed += create_bed(ccds)

ccds_bedtool = pybedtools.BedTool(ccds_bed, from_string=True)
ccds_bedtool_remove_invalid = ccds_bedtool.remove_invalid().sort()


In [46]:
ccds_bedtool.to_dataframe().shape

(30261, 6)

In [47]:
ccds_bedtool_remove_invalid.to_dataframe().shape

(30261, 6)

In [48]:
ccds_bedtool_remove_invalid.to_dataframe().drop_duplicates().shape

(30261, 6)

In [49]:
ccds_bedtool_remove_invalid.saveas('mm10_CCDS.bed.gz')

<BedTool(mm10_CCDS.bed.gz)>

In [50]:
ccds_bedtool_remove_invalid.saveas('mm10_CCDS.bed')

<BedTool(mm10_CCDS.bed)>

In [51]:
ccds_bedtool_remove_invalid.to_dataframe().drop_duplicates().sort_values(by=['chrom', 'start', 'end', 'strand']).to_csv('mm10_CCDS.bed', header=False, index=False, sep='\t')

# Exon level

In [53]:
gene_wise_CCDS_dict = DefaultOrderedDict(list)

for line_no, feature in enumerate(db.all_features()):
    gene_ids = feature.attributes['gene_id']    
    feature_type = feature.featuretype
    try:
        tags = feature.attributes['tag']
        if 'CCDS' not in tags:
            # Anything that is not CCDS is not interesting
            continue
        if feature_type != 'exon':
            continue
        else:
            assert len(gene_ids) == 1
            gene_id = gene_ids[0]
            transcript_ids = feature.attributes['transcript_id']
            assert len(transcript_ids) == 1
            transcript_id = transcript_ids[0]

            feature.attributes['gene_id'] = '{}_{}_exon{}'.format(gene_id, transcript_id, len(gene_wise_CCDS_dict[gene_id]))
            gene_wise_CCDS_dict[gene_id].append(feature)
    except:
        continue

In [54]:
ccds_bed = ''

for gene_id in get_gene_list(gene_dict):
    ccds = gene_wise_CCDS_dict[gene_id]
    ccds = list(sorted(list(ccds), key=lambda x: x.start))    
    ccds_bed += create_bed(ccds)

ccds_bedtool = pybedtools.BedTool(ccds_bed, from_string=True)
ccds_bedtool_remove_invalid = ccds_bedtool.remove_invalid().sort()


In [55]:
ccds_bedtool.to_dataframe().shape

(306986, 6)

In [56]:
ccds_bedtool_remove_invalid.to_dataframe().shape

(306986, 6)

In [57]:
ccds_bedtool_remove_invalid.to_dataframe().drop_duplicates().sort_values(by=['chrom', 'start', 'end', 'strand']).to_csv('mm10_CCDS_exon_level.bed', header=False, index=False, sep='\t')